In [1]:
# Import library yang diperlukan
import pandas as pd
import numpy as np
import pickle
from sklearn.model_selection import train_test_split
from sklearn.ensemble import VotingRegressor
from sklearn.metrics import mean_squared_error, r2_score
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor

In [ ]:
# Muat dataset
data = pd.read_csv('all.csv')  # Ganti dengan path dataset yang sesuai
data

ImportError: Unable to find a usable engine; tried using: 'pyarrow', 'fastparquet'.
A suitable version of pyarrow or fastparquet is required for parquet support.
Trying to import the above resulted in these errors:
 - Missing optional dependency 'pyarrow'. pyarrow is required for parquet support. Use pip or conda to install pyarrow.
 - Missing optional dependency 'fastparquet'. fastparquet is required for parquet support. Use pip or conda to install fastparquet.

In [15]:
# Pastikan kolom host_response_rate bertipe string
data['host_response_rate'] = data['host_response_rate'].astype(str)

# Hapus tanda '%' dan ubah menjadi tipe float
data['host_response_rate'] = data['host_response_rate'].str.rstrip('%').astype('float') / 100.0


In [16]:
# Siapkan variabel X dan y
X = data[['accommodates', 'bedrooms', 'minimum_nights', 'maximum_nights',
          'review_scores_rating', 'review_scores_cleanliness',
          'review_scores_location', 'host_total_listings_count', 'host_response_rate']]
y = data['price']


In [17]:
# Bagi data menjadi set latih dan uji
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=43)

In [18]:
# Buat model XGBoost dan LightGBM
xgb_model = XGBRegressor(n_estimators=100, random_state=43)
lgbm_model = LGBMRegressor(n_estimators=100, random_state=43)

In [19]:
# Gabungkan model menggunakan Voting Regressor
ensemble_model = VotingRegressor(estimators=[('xgb', xgb_model), ('lgbm', lgbm_model)])


In [20]:
# Latih model
xgb_model.fit(X_train, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=100, n_jobs=None,
             num_parallel_tree=None, random_state=43, ...)

In [21]:
# Evaluasi model pada set uji
y_pred = xgb_model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
rr = r2_score(y_test, y_pred)
print(f"Mean Squared Error: {mse}")
print(f"R-squared Score: {rr}")

Mean Squared Error: 2656.9554263167365
R-squared Score: 0.8467287957054137


In [22]:
# Simpan model dalam format .sav menggunakan pickle
with open(r"C:\Users\Aisya\Downloads\GelarRasa\Airbnb-Dashboard\airbnb_price_predictor_xgb.sav", 'wb') as model_file:
    pickle.dump(xgb_model, model_file)